In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
import ssl
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
import sys
sys.path.append('../')
from fun.models import *
sns.set_style("whitegrid")
from torch.utils.data import random_split
from tqdm import tqdm
import pandas as pd
torch.manual_seed(42)
np.random.seed(42)
import random
random.seed(42)


In [ ]:

transform = transforms.ToTensor()

dataset = torchvision.datasets.CIFAR10(
    root="../data",
    train=True,
    download=True,
    transform=transform  
)


torch.manual_seed(42)
valid_size = int(0.1 * len(dataset))
train_size = len(dataset) - valid_size


train_ds, valid_ds = random_split(dataset, [train_size, valid_size])
len(train_ds), len(valid_ds)

    

In [ ]:
def augment_data(dataset, tr1, tr2, tr3):
    dataset_augm = []
    len_ds = len(dataset)
    sample_to_aug1 = np.array(random.sample(range(len_ds), len_ds//5))
    sample_to_aug2 = np.array(random.sample(range(len_ds), len_ds//5))
    sample_to_aug3 = np.array(random.sample(range(len_ds), len_ds//5))

    for index, img in enumerate(dataset):
        #add original image
        dataset_augm.append(img)
        img_tensor = img[0]
        
        if index in sample_to_aug1:            
            img_tensor = tr1(img_tensor)
            #add augmented image
            dataset_augm.append((img_tensor, img[1]))

        elif index in sample_to_aug2:            
            img_tensor = tr2(img_tensor)
            #add augmented image
            dataset_augm.append((img_tensor, img[1]))

        elif index in sample_to_aug3:            
            img_tensor = tr3(img_tensor)
            #add augmented image
            dataset_augm.append((img_tensor, img[1]))
        
    return dataset_augm


In [ ]:
def augment_data_same_number(dataset, tr1, tr2, tr3):
    dataset_augm = []
    len_ds = len(dataset)
    sample_to_aug1 = np.array(random.sample(range(len_ds), len_ds//5))
    sample_to_aug2 = np.array(random.sample(range(len_ds), len_ds//5))
    sample_to_aug3 = np.array(random.sample(range(len_ds), len_ds//5))

    for index, img in tqdm(enumerate(dataset)):

        img_tensor = img[0]
        
        if index in sample_to_aug1:            
            img_tensor = tr1(img_tensor)

        if index in sample_to_aug2:            
            img_tensor = tr2(img_tensor)

        if index in sample_to_aug3:            
            img_tensor = tr3(img_tensor)
            
        dataset_augm.append((img_tensor, img[1]))
        

    return dataset_augm


In [ ]:
torchvision_transform1 = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((34, 34)), 
    transforms.RandomCrop(32),
    transforms.ToTensor(),
])

torchvision_transform2 = transforms.Compose([
    transforms.ToPILImage(),   
    transforms.Resize((28, 28)), 
    transforms.Pad(padding=2), 
    transforms.RandomHorizontalFlip(1),
    transforms.ToTensor(),
])

torchvision_transform3 = transforms.Compose([
    transforms.ToPILImage(),
    transforms.RandomRotation(degrees=45),
    transforms.ToTensor(),
])



In [ ]:
dataset_augm = augment_data(train_ds, torchvision_transform1,torchvision_transform2,torchvision_transform3)

In [ ]:
dataset_augm_same= augment_data_same_number(train_ds, torchvision_transform1,torchvision_transform2,torchvision_transform3)


In [ ]:

labels_map = {
    0:'plane', 
    1:'car', 
    2:'bird', 
    3:'cat',
    4:'deer', 
    5:'dog',
    6:'frog',
    7:'horse', 
    8:'ship', 
    9:'truck'

}
figure = plt.figure(figsize=(8, 8))
cols, rows = 5, 5
for i in range(1, cols * rows + 1):
    sample_idx = torch.randint(len(dataset_augm_same), size=(1,)).item()
    img, label = dataset_augm_same[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(labels_map[label])
    plt.axis("off")
    img = img.numpy()
    img = np.transpose(img, (1, 2, 0))
    plt.imshow(img.squeeze(), cmap="gray")
plt.show()

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

def train(dataloader, model, loss_fn, optimizer):

    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.train()
    train_loss, correct = 0, 0
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        
        pred = model(X)
        loss = loss_fn(pred, y)

       
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        train_loss += loss.item()
        correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    train_loss /= num_batches
    correct /= size
    print(f" Train Accuracy: {(100*correct):>0.1f}%, Train Avg loss {train_loss:>8f} \n")
    return correct, train_loss


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size

    print(f"Test Accuracy: {(100*correct):>0.1f}%, Test Avg loss: {test_loss:>8f} \n")
    return correct, test_loss
    

In [ ]:
#------------------------------------------------------------------
#--------------------- DO PUSZCZENIA ------------------------------
#------------------------------------------------------------------
lr = 1e-3
batch_size = 64
epochs = 60


data = np.array([[0]*4]*epochs)
net_SGD_augm_history = pd.DataFrame(data, columns = ["test_acc", "train_acc", "test_loss", "train_loss"])

train_dataloader = DataLoader(
        train_ds,
        batch_size=batch_size, 
        shuffle=True
        )
test_dataloader = DataLoader(
    valid_ds, 
    batch_size=batch_size,
    shuffle=False
    )

net = Net_dropout_new(dropout_rate=0.2).to(device)
criterion = nn.CrossEntropyLoss()



optimizer = torch.optim.Adam(net.parameters(), lr=lr)

for t in range(epochs):

    print(f"Epoch {t+1}\n-------------------------------")
    train_acc, train_loss = train(train_dataloader, net, criterion, optimizer)
    test_acc, test_loss = test(test_dataloader, net, criterion)

    net_SGD_augm_history.loc[t,"train_acc"] = train_acc        
    net_SGD_augm_history.loc[t,"test_acc"] = test_acc
    net_SGD_augm_history.loc[t,"train_loss"] = train_loss
    net_SGD_augm_history.loc[t,"test_loss"] = test_loss
        

        

In [ ]:
net_SGD_augm_history.to_csv(r"C:\Users\aleks\Desktop\DS\SEM_2\DL\Project1\DeepLearning\data\results\Net_dropout_new\adam_bs64_dr20\bnet_augm_history.csv")